In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# !unzip gdrive/MyDrive/data.zip > /dev/null

# Cat-Dog classification using transfer learning
###### Charlie Rosander 2023-09
A while back I made a Cat-Dog-CNN project, which was a simple CNN model classifying cats and dogs. Running 100 Epochs I would get around 80-85% accuracy, so now I want to try the same dataset but with transfer learning, to see if I can get a better result with fewer epochs etc. We will be using MobileNetV2 as our base model.

The dataset is from Kaggle: https://www.microsoft.com/en-us/download/details.aspx?id=54765

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications import MobileNetV2
import shutil
import pandas as pd
from numba import jit, cuda

We start by preparing some variables.

In [ ]:
# Preparing directory vars
base_dir = os.getcwd() + '/data'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

print(f"""
{base_dir},
{train_dir},
{test_dir}
""")

# Prepare image parameter vars
BATCH_SIZE = 32
IMG_SIZE = (96, 96)


### Data preprocessing
Here we use the ImageDataGenerator to load the images from the directories. I have split the data up into train, val and test in the directories, each with their respective subfolders (cat/dog).

We are also using MobileNetV2's preprocessing method as the preprocessing_function.
ImageDataGenerator automatically labels the images into classes based on the subfolders.

In [ ]:
# Preparing the preprocess method
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# Create an ImageDataGenerator object with rescaling and validation split
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2  # 20% for validation
)

# Create a test ImageDataGenerator object only with rescaling
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Training data: 80% from train directory
train_gen = train_datagen.flow_from_directory(train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training"  # set as training data
)

# Validation data: 20% from train directory
val_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation"  # set as validation data
)

# Test data: Take all images from test directory
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)


### Inspecting the data
We will inspect the data a bit more to see that everything is in order.

In [ ]:
# Here we inspect the classes to verify that they are correct.

print("Training class indices:", train_gen.class_indices)
print("Validation class indices:", val_gen.class_indices)
print("Test class indices:", test_gen.class_indices)


In [ ]:
# Inspect batch size and shape
for image_batch, label_batch in train_gen:
    print("Image batch shape:", image_batch.shape)
    print("Label batch shape:", label_batch.shape)
    break

In [ ]:
# Inspecting the images. Looks absolutely fantastic. And kinda creepy.
x_batch, y_batch = next(train_gen)

plt.figure(figsize=(10, 10), facecolor='white')
for i in range(6):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(x_batch[i])
    plt.title(f"Class: {y_batch[i]}")
    plt.axis('off')


### Creating the model, using the base model MobileNetV2.
We will use the pretrained model MobileNetV2 as our base model, and we will not include the top layer as we will add our own classifier layer and train it ourselves, as well as freeze the base model so we don't have to retrain it.

In [ ]:
# Creating the base model MobileNetV2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [ ]:
# # Here we retrieve a batch of images and their labels from train_gen, 
# passes them through the base model to extract the features and then prints the shape

image_batch, label_batch = next(iter(train_gen))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
# Freezing the base model
base_model.trainable = False

In [ ]:
# Inspect the base model
base_model.summary()

### Classification head
We need to add a classification head to the model, and we will convert the features to a single 1280-element vector.

In [ ]:
# Converting the features to a single 1280-element vector per image

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
# Adding a prediction layer
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

### Putting it all together
We will now put all the parts together, as well as some last minute data-augmentation because I forgot to write it earlier in the code.

In [ ]:
# Some data augmentation
data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip(
    'horizontal'), tf.keras.layers.RandomRotation(0.2),])

inputs = tf.keras.Input(shape=(96, 96, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

model.summary()

We can see in the output above that we now have 1,281 trainable parameters, which is from the classification head we added. These will be trained, while the base model will be frozen.

In [ ]:
# Compile the model
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# Var for the number of epochs
epoch_num = 10

# Train the actual model
history = model.fit(train_gen, epochs=epoch_num, validation_data=val_gen)

### Plotting and evaluation


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:
# Test the model
test_loss, test_acc = model.evaluate(test_gen, verbose=2)
print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Predict the labels of the test set
y_pred = model.predict(test_gen)
y_pred = np.argmax(y_pred, axis=1)

# Get the true labels
y_true = test_gen.classes

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
sns.heatmap(cm, annot=True)


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
